In [1]:
import pandas as pd
import math
from bokeh.plotting import figure, show, output_file  
from bokeh.models import  Band, ColumnDataSource ,  Legend
from bokeh.layouts import gridplot  
from sqlalchemy import create_engine 
import warnings as wrn
import numpy as np
# Imports the necessary libraries and modules are required to execute the following function().

wrn.filterwarnings('ignore')  

class class_for_finding_best_result:
    
    def __init__(self, training_data_csv_file, ideal_function_csv_file, test_data_csv_file):
        """
        Initializing instance variable.
        This function receives in three filenames, the training data file and ideal function file, and the test data file all in CSV format   
        computes the best fit functions for each of the four training data columns, 
        shows the training data and best fit functions using Bokeh, 
        maps the test data to the best-fit functions, 
        and saves the results in a SQLite database.
        
        """
    
        self.training_data_csv_file = training_data_csv_file
        self.ideal_function_csv_file = ideal_function_csv_file
        
        # data from the ideal and train functions are read into pandas dataframes. 
        self.train_func_dataframe = pd.read_csv(training_data_csv_file)
        self.ideal_func_dataframe = pd.read_csv(ideal_function_csv_file)
   
        # create a list from scratch with the best suited function column names.             
        self.best_four_ideal_func_list = []           
      
    def finding_best_result(self):    
        'Calculating the best fit from ideal function for each column of the train data'
        
        self.plots = []
        
        # loop over each training data y column.              
        for y_train_col in range(1, 5):                    
            # to get the string name of the current y column                        
            training_func_y_columns = 'y' + str(y_train_col)  
            # create a list that is initialised to include the squared sum of y-deviations for each ideal function.           
            deviation_squared_each_ideal_func = [0] * 50      
            
            # loop over each ideal function   
            for y_ideal_col in range(50):             
                # to get the string name of the ideal function  y column      
                ideal_func_y_columns = 'y' + str(y_ideal_col + 1)        
                
                # find the deviation error using Least square    
                # determine the difference/deviation between the training data's y column and the ideal function.     
                deviation_between_train_ideal = self.train_func_dataframe[training_func_y_columns] - self.ideal_func_dataframe[ideal_func_y_columns] 
                
                # sum the deviation squared formulae
                deviation_squared_each_ideal_func[y_ideal_col] += sum(deviation_between_train_ideal**2)   
                #deviation_squared_each_ideal_func[y_ideal_col] += np.sqrt(np.mean(np.square(deviation_between_train_ideal)))
                #deviation_squared[j] += np.sqrt(np.mean(np.square(deviation)))
                
            # The index of the best-fitting function with the smallest deviation can be found.  
            minimum_index_best_fit_func = deviation_squared_each_ideal_func.index(min(deviation_squared_each_ideal_func))       
            
            # store the best four ideal function column name 
            best_four_ideal_func = self.ideal_func_dataframe.columns[minimum_index_best_fit_func + 1] 
            self.best_four_ideal_func_list.append(best_four_ideal_func) 
            
            # calculate the total squared error   
        
            sum_of_all_squared_error = deviation_squared_each_ideal_func[minimum_index_best_fit_func] 
            
             
            # Take a copy of four best ideal function to another variable
            self.best_four_ideal_func_list_copy = self.best_four_ideal_func_list
            
            # plot the training data and the best four ideal functions        
            p = figure(title = f'Train model {training_func_y_columns} vs Ideal function {best_four_ideal_func}. Total squared error={(round(sum_of_all_squared_error,2))}',
                       x_axis_label = 'x', 
                       y_axis_label = 'y'
                       )
            
            p.scatter(self.train_func_dataframe['x'], 
                      self.train_func_dataframe[training_func_y_columns], 
                      legend_label = 'Train',
                      fill_color = '#A70D2A', 
                      line_width = 1, 
                      color = 'red'
                      )
            
            p.line(self.ideal_func_dataframe['x'], 
                   self.ideal_func_dataframe[best_four_ideal_func], 
                   legend_label = 'Ideal', 
                   line_width = 2.5,
                   line_color = '#151B54'
                   )
                
            p.legend.border_line_width = 3
            p.legend.border_line_color = 'darkgrey'
            p.legend.border_line_alpha = 0.5
            p.legend.title = 'Train vs Ideal'
            p.background_fill_color = 'beige'
            p.border_fill_color = 'whitesmoke'
            p.add_layout(Legend(), 'right')
            self.plots.append(p)
            
        grid = gridplot(self.plots, ncols = 1, width = 800)
        output_file('train_and_ideal.html')
        show(grid)
        


class test_data_mapping_to_best_fit_func(class_for_finding_best_result):
    'The test_data_mapping_to_best_fit_func class is a subclass of the class_finding_best_result class'
    
    def __init__(self, training_data_csv_file, ideal_function_csv_file, test_data_csv_file):
        'Inherit instance variables from parent class'
        super().__init__(training_data_csv_file, ideal_function_csv_file, test_data_csv_file)
        
        self.test_data_dataframe = pd.read_csv(test_data_csv_file)
     
        # Determine the largest difference between the training data and the selected ideal functions.  
        self.max_deviation_between_train_selected_ideal = {}

        
    def map_test_data_point(self):
        for y_train_col in range(1, 5):
            # to get the string name of the ideal function  y column
            train_func_y_columns = 'y' + str(y_train_col)
            
            for function in self.best_four_ideal_func_list_copy:
                # determine the difference between train and ideal then take a max value
                deviation_between_train_ideal = abs(self.train_func_dataframe[train_func_y_columns] - self.ideal_func_dataframe[function]).max()
                self.max_deviation_between_train_selected_ideal[function] = deviation_between_train_ideal
        
        # Mapping test data to selected best four ideal functions and initalize an empty list
        self.mapped_data_first = []
        self.mapped_data_first_copy = [] 
        
        # loop over each test data row.   
        for index, row in self.test_data_dataframe.iterrows():
            # extract the values of x and y from the row.  
            x, y = row['x'], row['y']
        
            # Initalize the list for store the difference values in deviations_list
            self.deviations_list = []
            
            # Iterate over each function   
            for function in self.best_four_ideal_func_list_copy:
                
                # calculate the y value using given x value and function   
                calculated_y_ideal_func_dataframe = self.ideal_func_dataframe[function].loc[self.ideal_func_dataframe['x'] == x].values[0]
                
                
                #alculated as the absolute difference between the calculated regression value and the corresponding ideal function value for a given x    
                # calculate the difference between calculated y value and actual y value    
                deviation_between_train_ideal = abs(calculated_y_ideal_func_dataframe - y)
                #print(deviation_between_train_ideal)
                # if the deviations is within the given conditions.
                if (deviation_between_train_ideal/self.max_deviation_between_train_selected_ideal[function]) <=   math.sqrt(2):
                    # add the func and difference to the created empty list     
                    self.deviations_list.append((function, deviation_between_train_ideal))
                
            
            # if only acceptable difference   
            if self.deviations_list:
                # sort the list by deviation values  
                sorted_deviations_list = sorted(self.deviations_list, key=lambda x: x[1])
                
                # store best function with the minimum deviation  
                best_function_value = sorted_deviations_list[0][0]
                
                # append the given values to mapped_data_first_copy list.  
                self.mapped_data_first_copy.append((x, y, best_function_value,sorted_deviations_list[0][1]))
                self.mapped_data_first.append((x, y, best_function_value))
                
        # Save the finding results to a SQLite database
        engine = create_engine('sqlite:///Mapping.db')
        conn = engine.connect()
        
        # Create the table 'Mapping' if the table doesn't exist
        conn.execute('''CREATE TABLE IF NOT EXISTS Mapping (
                            'X (test func)'   FLOAT,
                            'Y (test func)'   FLOAT,
                            'Delta Y (test func)'  FLOAT,
                            'No. of ideal func'    String(50))
                            ''')
        
        # Delete if any previous data stored in the table 'Mapping' in SQLite db
        conn.execute('''DELETE FROM Mapping''')
        
        # Vacuum the SQLite database
        conn.execute('''VACUUM''')
        
        
        
        # Insert the calculated data into the table 
        for x, y, best_function_value, deviation_between_train_ideal in self.mapped_data_first_copy:
            conn.execute(f"INSERT INTO Mapping ('X (test func)', 'Y (test func)', 'Delta Y (test func)', 'No. of ideal func') VALUES ({x}, {y}, {deviation_between_train_ideal}, '{best_function_value}')" )
    
        conn.close()
    
     
        
        # Visualize the data using Bokeh
        self.plots = []
    
        for x, y, best_function_value,deviation in self.mapped_data_first_copy:
            # Create a columndatasource for the test point values   
            data_source_test_func = ColumnDataSource(dict(x = [x], y = [y]))
            
            # create a columndatasource for the best four ideal function  
            data_source_ideal_func = ColumnDataSource(dict(x = self.ideal_func_dataframe['x'], y = self.ideal_func_dataframe[best_function_value]))
            
            # round the value to only two decimal places   
            round_y_col_value = round(y,2)
            
            # procedure of create to represents tolerance within the plot
            tolerance_graph =  self.max_deviation_between_train_selected_ideal[function] * math.sqrt(2)
            
            # caluclate the lower and upper bounds of the ideal functions   
            self.ideal_func_dataframe['upper'] = self.ideal_func_dataframe[best_function_value] + tolerance_graph
            self.ideal_func_dataframe['lower'] = self.ideal_func_dataframe[best_function_value] - tolerance_graph
    
            source_data_column = ColumnDataSource(self.ideal_func_dataframe.reset_index())
    
            band_range = Band(base = 'x', 
                              lower = 'lower', 
                              upper = 'upper', 
                              source = source_data_column, 
                              level = 'underlay',
                              fill_alpha = 0.3, 
                              line_width = 0.4, 
                              line_color = 'red', 
                              fill_color = "red"
                             )
            
            p = figure(title = f"Test Data Point ({x}, {round_y_col_value}) Mapped to {best_function_value}", 
                       x_axis_label = "x", 
                       y_axis_label = "y"
                      )
            
            p.circle(x = 'x', 
                     y = 'y', 
                     source = data_source_test_func, 
                     fill_color = '#A70D2A', 
                     line_color = '#A70D2A', 
                     line_width = 3, 
                     legend_label = 'Test data point', 
                     size = 12
                    )
            
            p.line(x = 'x', 
                   y = 'y', 
                   source = data_source_ideal_func,
                   legend_label = 'Classification function', 
                   line_width = 2.3, 
                   line_color = '#151B54'
                  )
    
            p.add_layout(band_range)
            p.legend.border_line_width = 2.9
            p.legend.border_line_color = 'darkgrey'
            p.legend.border_line_alpha = 0.6
            p.background_fill_color = 'beige'
            p.border_fill_color = 'whitesmoke'
            self.plots.append(p)
            
        grid = gridplot(self.plots, ncols = 1, width = 800)
        output_file('test_and_ideal.html')
        show(grid)
      
    
def main():
    best_fit_instance = test_data_mapping_to_best_fit_func(R"T:\Python\dataset\train.csv",R"T:\Python\dataset\ideal.csv",R"T:\Python\dataset\test.csv")
    best_fit_instance.finding_best_result()
    #best_fit_instance.plot_best_four_ideal_vs_train_data()
    best_fit_instance.map_test_data_point()

if __name__ == '__main__':
    main()


class sqlite_database_file_name:
    def __init__(self, csv_path_name):
        try:
            # _datas property is used to hold data from a read csv file..  
            self._datas = pd.read_csv(csv_path_name) 
            
        except FileNotFoundError:
            # Raising the error and printing a message if the file doesn't exist
            print('Mentioned pathname does not exist {}. Please check once again'.format(csv_path_name))
            raise
            
    def to_sql(self, db_file_name, last_word_col_name):
        
        # Create a SQLite database engine
        engine = create_engine('sqlite:///{}.db'.format(db_file_name), echo = False)
        
        # Take a data copy from the _datas using copy()
        data_first_copy = self._datas.copy()
        
        # Each column name should have its last word appended and its initial letter uppercase. 
        data_first_copy.columns = [name.capitalize() + last_word_col_name for name in data_first_copy.columns]
        
        # Set the first column as a index 
        data_first_copy.set_index(data_first_copy.columns[0], inplace = True)
        
        # Write the data to SQL db
        data_first_copy.to_sql (
            db_file_name, 
            engine, 
            if_exists = 'replace',  # If the table already exists, then replace its contents  
            index = True,  # Use the first column as the index in the SQLdatabase  
        )
        
# Create instances 
# Write the train.csv data to a SQLite database named 'Training_data'
train_instance = sqlite_database_file_name(R"T:\Python\dataset\train.csv")
training_database_table = train_instance.to_sql(db_file_name = 'Training_data', last_word_col_name = ' (training func)') 

# Write the ideal.csv data to a SQLite database named 'Ideal_function'
ideal_instance = sqlite_database_file_name(R"T:\Python\dataset\ideal.csv")
idealfunc_database_table = ideal_instance.to_sql(db_file_name = 'Ideal_function', last_word_col_name = ' (ideal func)') 

print('Program ran successfully!')

Program ran successfully
